# Project 

## Project Description

Dataset was last updated 1-8-2021.

## Part 1: Data Retrieval & Preprocessing


In [ ]:
# Import Libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import emoji

import nltk
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer


from datasets import load_dataset
import transformers
import simpletransformers
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModel,
)
from sklearn.metrics import classification_report


### once you downloaded vader successfully you do not need to do this again.
### You can command it out in your personal copy as I did below to skip this.

# nltk.download('vader_lexicon', quiet=False)

# Define file paths
sentiment_train_file = "Datasets/Sentiment_Analysis/twitter_training.csv"         
validation_file = "Datasets/Sentiment_Analysis/twitter_validation.csv" 
sentiment_test_file = "Datasets/Test Sets/sentiment-topic-test.tsv"

### Part 1.2 Sentiment Analysis Preprocessing

In [33]:
# Load the training data
sentiment_train_df = pd.read_csv(sentiment_train_file)
sentiment_validation_df= pd.read_csv(validation_file)

# Drop duplicates based on the 'ID' column
validation_data = sentiment_validation_df.drop_duplicates(subset='ID', keep='first')
sentiment_train_df = sentiment_train_df.drop_duplicates(subset='ID', keep='first')

# Remove rows with 'Irrelevant' in the third column
sentiment_train_df = sentiment_train_df[sentiment_train_df.iloc[:, 2] != 'Irrelevant']
sentiment_validation_df = sentiment_validation_df[sentiment_validation_df.iloc[:, 2] != 'Irrelevant']

# Rename the third column entries
sentiment_train_df.iloc[:, 2] = sentiment_train_df.iloc[:, 2].replace({
    'Positive': 'POS',
    'Neutral': 'NEU',
    'Negative': 'NEG'
})

# Save the updated training data back to the file
sentiment_train_df.to_csv(sentiment_train_file, index=False)
sentiment_validation_df.to_csv(validation_file, index=False)


In [34]:
# Load the model
sentimentenglish = pipeline("sentiment-analysis", 
                            model="finiteautomata/bertweet-base-sentiment-analysis", 
                            tokenizer="finiteautomata/bertweet-base-sentiment-analysis")

In [35]:
sentence_pos_en = "Nice hotel and the service is great"

sentimentenglish(sentence_pos_en)

[{'label': 'POS', 'score': 0.9918237328529358}]